In [8]:
#import data

# -*- coding: utf-8 -*-

#import data
from datetime import datetime
import pandas as pd
import numpy as np
import torch.onnx
import os

import torch
import torch.nn as nn
# import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets, transforms



In [9]:
all_x_data = []
all_y_data = []

file_path = os.path.join("..","dataset","traindata","drive_data_train.csv")
file_path = os.path.abspath(file_path)
print("===trin data file path===", "\n", file_path)

# Pandas를 이용해 파일 읽기
data = pd.read_csv(file_path, sep=",", header=0, dtype=float)
x_data = torch.tensor(data.iloc[:, [0,2,3,4,5]].values, dtype=torch.float32)
y_data = torch.tensor(data.iloc[:, 1:2].values, dtype=torch.float32)

# 전체 데이터 리스트에 추가
all_x_data.append(x_data)
all_y_data.append(y_data)
        
# 리스트에 저장된 모든 데이터를 하나의 텐서로 결합
all_x_data = torch.cat(all_x_data, dim=0)
all_y_data = torch.cat(all_y_data, dim=0)

# 전체 데이터에 대한 평균과 분산 계산
mean_x = torch.mean(all_x_data, dim=0)
std_x = torch.std(all_x_data, dim=0)

Max_Vy = max(abs(all_y_data))

===trin data file path=== 
 c:\github_JM\2024-Neural-networks-and-deep-learning-teamproject\dataset\traindata\drive_data_train.csv


In [10]:
x_data = (all_x_data - mean_x) / std_x  # mean과 std는 훈련 데이터셋에서 계산된 값 사용
y_data = all_y_data/float(Max_Vy)

# 배치사이즈 설정
batch_size = 512

In [11]:
class My_dataset(Dataset):
    def __init__(self, x_data, y_data, transform=None):  #데이터 셋 선처리 해주는 부분
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):  # 데이터 셋 한 개의 데이터를 가져오는 함수 정의하는 부분

        if isinstance(index, slice):
            # 슬라이스 처리
            start_index = index.start
            end_index = index.stop
        else:
            # 단일 인덱스 처리
            start_index = index
            end_index = start_index + batch_size

        end_index = min(end_index, self.len)
        x = self.x_data[start_index:end_index]
        y = self.y_data[start_index:end_index]

        if self.transform:
            x, y = self.transform((x, y))
            sample = x, y
        return sample

    def __len__(self):  # 데이터 셋의 길이 적어 주는 부분
        return self.len - batch_size + 1


class ToTensor:
    def __call__(self, sample):
        x, y = sample
        x = torch.FloatTensor(x)
        y = torch.FloatTensor(y)
        return x, y


# 데이터 로드
trans = transforms.Compose([ToTensor()])

train_data = My_dataset(x_data, y_data, transform=trans)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

In [12]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(6, 64)
        self.fc2 = torch.nn.Linear(64, 64)
        self.fc3 = torch.nn.Linear(64, 1)
        # self.fc4 = torch.nn.Linear(1, 1)
        self.ln1 = torch.nn.LayerNorm(64)
        self.ln2 = torch.nn.LayerNorm(64)
        # self.ln3 = torch.nn.LayerNorm(32)
        self.dropout = torch.nn.Dropout()
        self.gelu = torch.nn.GELU()

    def forward(self, x):
        x = self.gelu(self.ln1(self.fc1(x)))
        x = self.dropout(x)
        x = self.gelu(self.ln2(self.fc2(x)))
        x = self.dropout(x)
        # x = self.gelu(self.ln3(self.fc3(x)))
        # x = self.dropout(x)
        x = torch.tanh(self.fc3(x))
        return x

# model = Net().to(device)
model = Net()
Loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.00005)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.99)

In [13]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

4929


In [14]:
now = datetime.now()
formatted_time = now.strftime("%Y_%m_%d_%H_%M")

model_path = os.path.join("model","RNN_model_{0}.pt".format(formatted_time))
model_path = os.path.abspath(model_path)
print("===trained model path===", "\n", model_path)

losses = []

# 훈련 루프
num_epochs = 600
num_data = len(x_data)

batch_predictions = torch.zeros(batch_size, 1)

min_loss = 50
training_epoch = 0
try:
    for epoch in range(num_epochs):
        losses = []
        epoch_losses = []
        average_losses = []
        prev_targets = torch.zeros(1,1)
    
        for i in range(0, num_data - batch_size + 1, batch_size):
            inputs, targets = train_data[i:i + batch_size]
            
            optimizer.zero_grad()
            
            batch_prediction_list = []
            current_batch_size = inputs.size(0)
            prev_predictions = torch.zeros(1, 1)
            prev_targets = prev_targets
    
            for j in range(batch_size):
                zero_predictions = torch.zeros(1, 1)
                # bank_err_1 = inputs[j, 1] - prev_predictions
    
    
                if i == 0 and j == 0:
                    bank_err_1 = inputs[j, 1] - zero_predictions
                    step_inputs = torch.cat([inputs[j].unsqueeze(0), zero_predictions], dim=1)
    
                elif i >= 1 and j == 0:
                    # bank_err_1 = inputs[j, 0] - prev_predictions
                    step_inputs = torch.cat([inputs[j].unsqueeze(0), prev_targets], dim=1)
    
                else:
                    step_inputs = torch.cat([inputs[j].unsqueeze(0), prev_targets], dim=1)
                    # print("step_inputs:",step_inputs.size())
    
                step_outputs = model(step_inputs)
                prev_targets = targets[j-1].unsqueeze(0)
                
                prev_predictions = step_outputs
    
    
                batch_prediction_list.append(step_outputs.squeeze())
    
            batch_predictions = torch.stack(batch_prediction_list).view(-1, 1)
    
            loss = Loss(batch_predictions, targets)
    
            epoch_losses.append(loss.item())
    
            # 역전파 및 최적화
            
            loss.backward()
            optimizer.step()
    
        average_losses = np.mean(epoch_losses)
        losses.append(average_losses)
    
        if average_losses < min_loss:
            min_loss = average_losses
            model.eval()  # 모델 평가 모드로 전환
            x_dummy = torch.randn(1, 6)
            traced_model = torch.jit.trace(model, x_dummy)
            traced1_model_path = model_path[:-3] + '_Traced_1.pt'
            torch.jit.save(traced_model, traced1_model_path)

        if epoch in [100,200,300,400]:
            model.eval()  # 모델을 평가 모드로 설정
            x_dummy = torch.randn(1, 6)
            traced_model = torch.jit.trace(model, x_dummy)
            evaluated_model_path = model_path[:-3] + '_Evaluated_' + str(epoch) + '.pt'
            torch.jit.save(traced_model, evaluated_model_path)

        # Scheduler 업데이트
        scheduler.step()
        training_epoch = epoch
        # 진행 상황 출력 (선택적)
        print(f"Epoch {epoch}, average_losses: {average_losses}")        

except KeyboardInterrupt:
    print("Training interrupted. Saving the model.")
    model.eval()  # 모델을 평가 모드로 설정
    x_dummy = torch.randn(1, 6)
    traced_model = torch.jit.trace(model, x_dummy)
    # 트레이싱된 모델 저장
    traced2_model_path = model_path[:-3] + '_Traced_2.pt'
    torch.jit.save(traced_model, traced2_model_path)

# 모델 저장
model.eval()  # 모델을 평가 모드로 설정
x_dummy = torch.randn(1, 6)
traced_model = torch.jit.trace(model, x_dummy)

# 트레이싱된 모델 저장
traced3_model_path = model_path[:-3] + '_Traced_3.pt'
torch.jit.save(traced_model, traced3_model_path)
print("Traced model saved.")

===trained model path=== 
 c:\github_JM\2024-Neural-networks-and-deep-learning-teamproject\RNN\model\RNN_model_2024_06_05_19_48.pt
Training interrupted. Saving the model.
Traced model saved.
